In [ ]:
from typing import List
import os
import pandas as pd
from tqdm.notebook import tqdm
import datasets

In [ ]:
DATA_DIR = 'crisis_data'
FILE_BLACKLIST = [
    'crisis_data/Afera Rywina.xlsx',
    'crisis_data/Ministerstwo Zdrowia_respiratory od handlarza bronią.xlsx',
    'crisis_data/Fake news_baza publikacji.xlsx'
]

files = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f[-5:] == '.xlsx']
for f in FILE_BLACKLIST:
    files.remove(f)

In [ ]:
def create_dataset(filenames: List[str]):
    dss = []
    for filename in tqdm(filenames):
        df = pd.read_excel(filename)
        df = df[['Kryzys', 'Tytuł publikacji', 'Lead', 'Kontekst publikacji']]

        if df['Kryzys'].hasnans:
            df['Kryzys'] = df['Kryzys'].notna()
        else:
            df['Kryzys'] = (df['Kryzys'] != 'NIE') & (df['Kryzys'] != 'Nie')
        if df['Kryzys'].nunique() != 2:
            raise RuntimeError(f'Crisis column data error in file {filename}.')

        df['text'] = df.apply(lambda x: ".".join([str(x['Tytuł publikacji']), str(x['Lead']), str(x['Kontekst publikacji'])]), axis=1)
        df = df[['Kryzys', 'text']]
        dss.append(datasets.Dataset.from_pandas(df))
    return datasets.concatenate_datasets(dss)